## Objective:
    
To make a theory calculation for the 2D (angular) cross correlation for the PanSTARRS Supernova and BOSS (CMASS & LOWZ) galaxy data sets, that was found previously for comparison and validation.

### Notes: 

- NIntegrate Chi integral in Mathematica

Author: Megan Tabbutt <br/>
Last Revised: 8_7_20

### Calculations:

_(2D) Cross Correlation Function (from Ross):_

$w(\theta)^{gg}_{ur} = b_u b_r \; \int \; n_u(z) n_r(z) \; \frac{dz}{d\chi} \frac{dz}{d\chi} \; d\chi \; \int \frac{k}{2 \pi} \; P(k, z(\chi)) \; J_0(\chi k \theta) \;dk $

Comoving distance formula from Dodelson:

$\chi(a) = \int^1_a \frac{da}{a^2H(a)}$ with $a = \frac{1}{1+z}$, $\chi(z) = \int^z_0 \frac{dz}{H(z)}$

and 

$H(z)^2 = H_0^2(\Omega_m(1+z)^3 + \Omega_k(1+z)^2 + \Omega_{\Lambda})$ (Wikipedia - Hubble Law)

In [12]:
# Working on 2015 laptop: Need to be in CCL conda environment to work on CAMB
import camb
import numpy as np
import pandas as pd
import sqlite3
import scipy as sp
import scipy.integrate

In [2]:
DATA_PATH = '/Users/megantabbutt/CosmologyDataProducts/'

In [19]:
# Define Cosmology:

Omega_m = .286
Omega_k = 0
Omega_L = .65193
hubble_0 = 69.6 # km/sec/Mpc
c = 3.0*(10**8) 

### Pull in and Parse Data:

In [3]:
qry_PanSTARRS_Data_All = "SELECT ID, DEC, RA, zSN, zHost FROM PanSTARRSNEW WHERE (zSN > -999) || (zHost > -999)"

connPAN = sqlite3.connect(DATA_PATH + 'PanSTARRS.db')
PanSTARRSNEW_GoodZ = pd.read_sql(qry_PanSTARRS_Data_All, con=connPAN)
connPAN.close()
PanSTARRSNEW_GoodZ.head(3) # 1129 objects over 10 pointings 

,ID,DEC,RA,zSN,zHost
0,ps0909006,1.184831,333.9500,0.284,-999.0
1,ps0909010,-4.078911,37.1182,0.270,-999.0
2,ps0910017,-5.023289,35.2775,0.320,-999.0


##### Note: Take the redshift of the supernova if both are given. It redshift of supernova is unknown, take the redshift of the host. 

In [4]:
zBest = []
for i in PanSTARRSNEW_GoodZ.iterrows(): #.iterrows() returns a tuple (immutable object)
    if i[1]['zSN'] >= 0:
        zBest.append(i[1]['zSN'])
    else:
        zBest.append(i[1]['zHost'])
        
if PanSTARRSNEW_GoodZ.shape[0] == len(zBest):
    PanSTARRSNEW_GoodZ['zBest'] = zBest
    
PanSTARRSNEW_GoodZ.head(3)

,ID,DEC,RA,zSN,zHost,zBest
0,ps0909006,1.184831,333.9500,0.284,-999.0,0.284
1,ps0909010,-4.078911,37.1182,0.270,-999.0,0.270
2,ps0910017,-5.023289,35.2775,0.320,-999.0,0.320


### Get chi(z) and interpolate to get z(chi):

In [21]:
hubble_z_inverse = lambda z: 1.0/(np.sqrt((hubble_0**2)*(Omega_m*(1 + z)**3 + Omega_k*(1 + z)**2 + Omega_L)))
# chi_z = scipy.integrate.quad(hubble_z_inverse, 0, zLimit)

chi_z = []
for i in zBest:
    zLimit = i
    chi_z_tup = scipy.integrate.quad(hubble_z_inverse, 0, zLimit)
    chi_z.append(chi_z_tup[0]*c)

print(chi_z)

[1178674.7621671348, 1124646.7991919213, 1315659.9145818949, 1105248.2568073825, 967940.5452561651, 1027112.1546125262, 1537789.939243514, 590183.7136052651, 1046727.5116701213, 644113.7123641605, 652378.2846643971, 1738232.8515636066, 1023182.5737979974, 438943.75772270566, 1390557.9340568723, 1501301.0943727354, 2345482.3072869396, 1822134.4989799785, 928221.437451955, 827973.8544527792, 1277888.6969365506, 392178.88346528815, 136986.95991764517, 544266.4361080102, 1642412.328045335, 967940.5452561651, 644113.7123641605, 1085795.5433546468, 1993135.5335075557, 928221.437451955, 1776835.683402365, 1541427.165671264, 1762827.315758729, 1046727.5116701213, 1159427.7501385591, 979813.9403189828, 1375646.9437307492, 1717070.849610568, 2060119.9312928983, 2126306.540229233, 2211169.5284350305, 1563206.0863008446, 1326949.344720755, 1464599.8604858671, 1464599.8604858671, 1416569.7831672423, 1356960.0004217958, 1787320.229263511, 1762827.315758729, 263334.1909783397, 1427685.656401991, 2892

In [ ]:
import scipy.integrate
from numpy import exp
f= lambda x:exp(-x**2)
i = scipy.integrate.quad(f, 0, 1)
print i